In [89]:
from sklearn import tree
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from statistics import stdev
from statistics import mean

df = pd.read_csv('../input/titanic/train.csv')

# Replace data in column Embarked C (Cherbourg) = 0, Q (Queenstown) = 1, S (Southampton) = 2
df['Embarked'] = df['Embarked'].replace(['C','Q','S'],[0, 1, 2])

# Replace data in column Sex male = 0, female = 1
df['Sex'] = df['Sex'].replace(['male','female'],[0, 1])

X = df.drop(['Survived','Name','Ticket','Cabin'], axis=1)
y = df['Survived']

# Calculo da média de cada coluna
media = X.mean(axis = 0, skipna = True)

# Preenchendo os valores NaN com as médias de cada coluna
X = X.fillna(value=media)

skf = StratifiedKFold(n_splits=7, shuffle=True)

acuracia = []
for train_index, test_index in skf.split(X,y):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    #print('train_index: ', train_index)
    #print('test_index: ', test_index)
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(X_train, y_train)
    yhat = clf.predict(X_test)
    result = accuracy_score(y_test, yhat)
    acuracia.append(result)
    
print('Acurácia: ', mean(acuracia),'(', stdev(acuracia),')')

Acurácia:  0.7508700084364455 ( 0.04951619603943639 )


In [96]:
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
import numpy
from sklearn.model_selection import train_test_split

acuracia = []
for train_index, test_index in skf.split(X,y):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    
    # create model
    model = Sequential()
    model.add(Dense(32, input_dim=X_train.shape[1], activation='relu',))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    #opt = SGD(learning_rate=0.001)
    opt = Adam(learning_rate=0.01)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

    history = model.fit(X_train, y_train, epochs=500, batch_size=32, verbose=0, callbacks=[es])

    _, test_acc = model.evaluate(X_test, y_test, verbose=0)
    acuracia.append(test_acc)
    
print('Acurácia: ', mean(acuracia),'(', stdev(acuracia),')')

Acurácia:  0.7565030966486249 ( 0.054723044623868296 )
